We have a 96 well plate containing 16 different compounds in the first two columns. Each well contains 2 µL of a stock solution at 1 mM. We want to carry out an end-point assay at 5 µM in triplicate in a 96 well plate with the addition of a second reagent. 

In [83]:
import itertools
import math
from itertools import islice
from pprint import pprint

import numpy as np
import opentrons.simulate

protocol = opentrons.simulate.get_protocol_api("2.12")
protocol.home()

%run opentrons_titrate_functions.ipynb

/Users/timkench/.opentrons/robot_settings.json not found. Loading defaults
/Users/timkench/.opentrons/deck_calibration.json not found. Loading defaults


In [84]:
tiprack_20a = protocol.load_labware("opentrons_96_tiprack_20ul", location="7")
tiprack_20b = protocol.load_labware("opentrons_96_tiprack_20ul", location="8")
tiprack_20c = protocol.load_labware("opentrons_96_tiprack_20ul", location="9")

tiprack_300a = protocol.load_labware("opentrons_96_tiprack_300ul", location="10")
tiprack_300b = protocol.load_labware("opentrons_96_tiprack_300ul", location="11")

m20 = protocol.load_instrument(
    "p20_multi_gen2", "left", tip_racks=[tiprack_20a, tiprack_20b, tiprack_20c]
)
m300 = protocol.load_instrument(
    "p300_multi_gen2", "right", tip_racks=[tiprack_300a, tiprack_300b]
)

labware_96w_200ul = "biorad_96_wellplate_200ul_pcr"
labware_12w_22ml = "nest_96_wellplate_2ml_deep"
labware_384w_20ul = "appliedbiosystemsmicroamp_384_wellplate_40ul"

res_plate_ot = protocol.load_labware(labware_12w_22ml, "1")
stock_plate_ot = protocol.load_labware(labware_96w_200ul, "2")
measure_plate_ot = protocol.load_labware(labware_96w_200ul, "3")

In [ ]:
# set up plates

res_plate = ResPlate(reagents={"buffer": 0, "reagent": 1})
res_plate.assign_ot(res_plate_ot)

stock_plate = GeneralPlate(active_cols=(0, 2), start_conc=1000, start_vol=2)
stock_plate.assign_ot(stock_plate_ot)

measure_plate = MeasurePlate(map_to=stock_plate, repeats=3)
measure_plate.assign_ot(measure_plate_ot)
measure_plate.make_transfer_map()

# set up pipettes and protocol

m20_c = CustomPipette(m20, name="m20_c")
m300_c = CustomPipette(m300, name="m300_c")

CP = CustomProtocol(
    solvent_location=res_plate.reagents["buffer"], m20=m20_c, m300=m300_c
)

# do titration

CP.add(plate=stock_plate, vol=200)
CP.transfer(plate=measure_plate, vol=50)
CP.add(
    plate=measure_plate,
    source=res_plate.reagents["reagent"],
    vol=50,
    top_dispense="Yes",
)

This will add the specified source to the active columns of the plate using the m300_c pipette.
The plate will be diluted to 10.0 µM


In [82]:
for line in protocol.commands():
    print(line)

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Aspirating 216.0 uL from A1 of NEST 96 Deep Well Plate 2mL on 1 at 94.0 uL/sec
Delaying for 0 minutes and 0.0 seconds
Moving to A1 of NEST 96 Deep Well Plate 2mL on 1
Dispensing 200.0 uL into A1 of Bio-Rad 96 Well Plate 200 µL PCR on 2 at 94.0 uL/sec
Delaying for 0 minutes and 0.0 seconds
Moving to A1 of Bio-Rad 96 Well Plate 200 µL PCR on 2
Dispensing 17.6 uL into A1 of NEST 96 Deep Well Plate 2mL on 1 at 94.0 uL/sec
Delaying for 0 minutes and 0.0 seconds
Moving to A1 of NEST 96 Deep Well Plate 2mL on 1
Aspirating 216.0 uL from A1 of NEST 96 Deep Well Plate 2mL on 1 at 94.0 uL/sec
Delaying for 0 minutes and 0.0 seconds
Moving to A1 of NEST 96 Deep Well Plate 2mL on 1
Dispensing 200.0 uL into A2 of Bio-Rad 96 Well Plate 200 µL PCR on 2 at 94.0 uL/sec
Delaying for 0 minutes and 0.0 seconds
Moving to A2 of Bio-Rad 96 Well Plate 200 µL PCR on 2
Dispensing 15.84 uL into A1 of NEST 96 Deep Well Plate 2mL on 1 at 94.0 uL/sec
Delayi